In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import GradientBoostingClassifier



data = pd.read_csv('./data/mental_health_and_technology_usage_2024.csv')

# Первичный анализ дата сета


### Посмотрим на первые 5 строк дата сета

In [ ]:
data.head()

In [ ]:
data.shape

### Посмотрим на размер дата сета  и типы данных

In [ ]:
data.info()


### Посмотрим на статистику дата сета

In [ ]:
data.describe()

### Посмотрим на количество пропущенных значений

In [ ]:
data.isnull().sum()

### Посмотрим на количество дубликатов

In [ ]:
print(data.duplicated().sum())

### Посмотрим на количество уникальных значений

In [ ]:
data.nunique()

In [ ]:
data['Age'].unique()

### Посмотрим на распределение значений в колонке Age

In [ ]:
data['Age'].value_counts()

In [ ]:
data['Age'].describe()

# Анализ данных

### Функция для визуализации распределения признака


In [ ]:
def plot_distribution(data, feature):
    plt.figure(figsize=(10, 6))
    if data[feature].dtype in ['int64', 'float64']:
        sns.histplot(data[feature], kde=True)
        plt.title(f'Distribution of {feature}')
    else:
        sns.countplot(y=feature, data=data)
        plt.title(f'Distribution of {feature}')
        plt.ylabel('')
    plt.tight_layout()
    plt.show()

### Визуализация распределения признаков


In [ ]:
features_to_visualize = [col for col in data.columns if col != 'User_ID']
for feature in features_to_visualize:
    plot_distribution(data, feature)


### Корреляционный анализ для числовых признаков


In [ ]:
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
correlation_matrix = data[numeric_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix for Numeric Features')
plt.tight_layout()
plt.show()


### Функция для проведения теста хи-квадрат


In [ ]:
def test_uniformity(data, feature, p_value=0.05):
    if data[feature].dtype in ['int64', 'float64']:
        # Для числовых признаков используем биннинг
        _, bin_edges = np.histogram(data[feature], bins='auto')
        observed, _ = np.histogram(data[feature], bins=bin_edges)
        n = len(data[feature])
        expected = np.array([n / len(observed)] * len(observed))
    else:
        # Для категориальных признаков используем частоты категорий
        observed = data[feature].value_counts().values
        n = len(data[feature])
        expected = np.array([n / len(observed)] * len(observed))

    chi2, p = stats.chisquare(observed, expected)

    return {
        'feature': feature,
        'chi2_statistic': chi2,
        'p_value': p,
        'is_uniform': p > p_value
    }

### Проведение теста хи-квадрат для всех признаков
- H0: Распределение значений признака равномерно
- H1: Распределение значений признака не равномерно
- Уровень значимости: 0.05

In [ ]:
# Проводим тест для всех признаков, кроме User_id
features_to_test = [col for col in data.columns if col != 'User_ID']
results = []

for feature in features_to_test:
    result = test_uniformity(data, feature)
    results.append(result)

In [ ]:
# Создаем датафрейм с результатами
results_df = pd.DataFrame(results)

# Сортируем результаты по p-value
results_df = results_df.sort_values('p_value')

results_df

### Визуализация результатов теста хи-квадрат


In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=results_df, x='chi2_statistic', y='p_value', hue='is_uniform', style='is_uniform')
plt.title('Результаты теста на равномерность распределения')
plt.xlabel('Статистика хи-квадрат')
plt.ylabel('p-value')
plt.axhline(y=0.05, color='r', linestyle='--', label='Уровень значимости (0.05)')
plt.legend(title='Равномерное распределение')
plt.tight_layout()
plt.show()

### Функция для визуализации зависимостей между числовыми и категориальными признаками


In [ ]:
def plot_boxplot(data, num_feature, cat_feature):
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=cat_feature, y=num_feature, data=data)
    plt.title(f'{num_feature} by {cat_feature}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


### Визуализация зависимостей между числовыми и категориальными признаками


In [ ]:
categorical_features = data.select_dtypes(include=['object']).drop('User_ID', axis=1).columns
for num_feature in numeric_features:
    for cat_feature in categorical_features:
        plot_boxplot(data, num_feature, cat_feature)

### Проверка равномерности зависимостей между числовыми и категориальными признаками
- H0: Зависимость между числовым и категориальным признаком равномерна
- H1: Зависимость между числовым и категориальным признаком не равномерна
- Уровень значимости: 0.05

In [ ]:
def test_uniformity_numeric_vs_categorical(data, numeric_feature, categorical_feature, p_value=0.05):
    groups = [group for _, group in data.groupby(categorical_feature)[numeric_feature]]
    f_statistic, p = stats.f_oneway(*groups)

    df_between = len(groups) - 1
    df_within = sum(len(group) - 1 for group in groups)
    eta_squared = (df_between * f_statistic) / (df_between * f_statistic + df_within)
    effect_size = np.sqrt(eta_squared)

    return {
        'numeric_feature': numeric_feature,
        'categorical_feature': categorical_feature,
        'f_statistic': f_statistic,
        'p_value': p,
        'is_uniform': p > p_value,
        'effect_size': effect_size
    }

### Определяем числовые и категориальные признаки


In [ ]:
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = data.select_dtypes(include=['object']).columns

numeric_features = [col for col in numeric_features if col != 'User_ID']
categorical_features = [col for col in categorical_features if col != 'User_ID']

### Проводим тест для всех комбинаций числовых и категориальных признаков


In [ ]:
dependency_results = []

for num_feature in numeric_features:
    for cat_feature in categorical_features:
        result = test_uniformity_numeric_vs_categorical(data, num_feature, cat_feature)
        dependency_results.append(result)

# Создаем датафрейм с результатами
dependency_df = pd.DataFrame(dependency_results)
dependency_df

### Визуализация результатов теста для числовых и категориальных признаков

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(data=dependency_df, x='effect_size', y='p_value', hue='is_uniform', style='is_uniform')
plt.title('Результаты теста на равномерность зависимости числовых признаков от категориальных')
plt.xlabel('Размер эффекта (корень из эта-квадрат)')
plt.ylabel('p-value')
plt.axhline(y=0.05, color='r', linestyle='--', label='Уровень значимости (0.05)')
plt.legend(title='Равномерная зависимость')
plt.tight_layout()
plt.show()

### Функция для визуализации зависимости


In [ ]:
def plot_dependency(data, numeric_feature, categorical_feature):
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=categorical_feature, y=numeric_feature, data=data)
    plt.title(f'Зависимость {numeric_feature} от {categorical_feature}')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


### Визуализация наиболее значимых зависимостей


In [ ]:
top_dependencies = dependency_df[dependency_df['is_uniform'] == False].head(5)
for _, row in top_dependencies.iterrows():
    plot_dependency(data, row['numeric_feature'], row['categorical_feature'])

### Сравнение результатов с предыдущим анализом


In [ ]:
for feature in results_df['feature']:
    if feature in numeric_features:
        prev_result = results_df[results_df['feature'] == feature]['is_uniform'].values[0]
        curr_results = dependency_df[dependency_df['numeric_feature'] == feature]['is_uniform'].values

        if prev_result != all(curr_results):
            print(f"Несоответствие для признака {feature}:")
            print(f"  Предыдущий результат: {'равномерное' if prev_result else 'неравномерное'} распределение")
            print(f"  Текущий результат: {'все' if all(curr_results) else 'не все'} зависимости равномерны")


### Анализ влияния категориальных признаков


In [ ]:
category_impact = dependency_df.groupby('categorical_feature').agg({
    'is_uniform': lambda x: (~x).mean(),
    'effect_size': 'mean'
}).sort_values('is_uniform', ascending=False)

category_impact

### Визуализация влияния категориальных признаков


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(data=category_impact.reset_index(), x='effect_size', y='is_uniform', s=100)
for i, row in category_impact.reset_index().iterrows():
    plt.annotate(row['categorical_feature'], (row['effect_size'], row['is_uniform']))
plt.title('Влияние категориальных признаков на распределение числовых')
plt.xlabel('Средний размер эффекта')
plt.ylabel('Доля неравномерных зависимостей')
plt.tight_layout()
plt.show()

# Выводы по Анализу данных

## Визуализация распределения признаков:
* Создана функция `plot_distribution` для визуализации распределения каждого признака.
* Построены гистограммы для числовых признаков и столбчатые диаграммы для категориальных.
* Визуализация позволила быстро оценить характер распределения каждого признака.

## Корреляционный анализ:
* Построена тепловая карта корреляций для числовых признаков.
* Использована библиотека seaborn для наглядного представления корреляционной матрицы.
* Анализ позволил выявить наиболее сильные линейные связи между признаками.

## Тест на равномерность распределения:
* Разработана функция `test_uniformity` для проведения теста хи-квадрат.
* Проведен тест для всех признаков с уровнем значимости 0.05.
* Результаты теста визуализированы с помощью scatter plot, где показаны статистика хи-квадрат и p-value для каждого признака.
* Выявлены признаки с неравномерным распределением.

## Анализ зависимостей между числовыми и категориальными признаками:
* Создана функция `plot_boxplot` для визуализации зависимостей.
* Построены box plot'ы для всех комбинаций числовых и категориальных признаков.
* Разработана функция `test_uniformity_numeric_vs_categorical` для проведения однофакторного дисперсионного анализа (ANOVA).
* Проведен тест для всех комбинаций числовых и категориальных признаков.
* Результаты визуализированы с помощью scatter plot, показывающего размер эффекта и p-value для каждой комбинации.

## Сравнение результатов с предыдущим анализом:
* Проведено сравнение результатов теста на равномерность распределения с результатами анализа зависимостей.
* Выявлены несоответствия в результатах для некоторых признаков.

## Анализ влияния категориальных признаков:
* Рассчитана доля неравномерных зависимостей для каждого категориального признака.
* Вычислен средний размер эффекта для каждого категориального признака.
* Результаты визуализированы с помощью scatter plot, показывающего влияние каждого категориального признака на распределение числовых.

## Выводы:
* Выявлены признаки с наиболее неравномерным распределением.
* Обнаружены значимые зависимости между некоторыми числовыми и категориальными признаками.
* Определены категориальные признаки, оказывающие наибольшее влияние на распределение числовых (например, "Work_Environment_Impact").



# Обучение модели


### Игнорируем предупреждения о сходимости

In [40]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

### Определение функций 

In [56]:
def preprocess_data(df):
    numeric_features = ['Age', 'Technology_Usage_Hours', 'Social_Media_Usage_Hours',
                        'Gaming_Hours', 'Screen_Time_Hours', 'Sleep_Hours', 'Physical_Activity_Hours']
    categorical_features = ['Gender', 'Stress_Level', 'Support_Systems_Access',
                            'Work_Environment_Impact', 'Online_Support_Usage']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', Pipeline([
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
            ]), numeric_features),
            ('cat', Pipeline([
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=bool))
            ]), categorical_features)
        ])

    return preprocessor

def create_and_train_model(preprocessor, model, X_train, y_train, param_grid):
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)

    return grid_search

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    result = {
        'Model': model.best_estimator_['classifier'].__class__.__name__,
        'F1 Score': f1_score(y_test, y_pred, average='weighted'),
        'Recall': recall_score(y_test, y_pred, average='weighted'),
        'Precision': precision_score(y_test, y_pred, average='weighted')
    }
    result.update({f'param_{k}': v for k, v in model.best_params_.items() if k.startswith('classifier__')})
    return result

def plot_feature_importance(model, feature_names):
    if hasattr(model.best_estimator_['classifier'], 'feature_importances_'):
        importances = model.best_estimator_['classifier'].feature_importances_
        indices = np.argsort(importances)[::-1]
        plt.figure(figsize=(10, 6))
        plt.title("Важность признаков")
        plt.bar(range(len(importances)), importances[indices])
        plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=90)
        plt.tight_layout()
        plt.show()
    elif hasattr(model.best_estimator_['classifier'], 'coef_'):
        importances = np.abs(model.best_estimator_['classifier'].coef_[0])
        indices = np.argsort(importances)[::-1]
        plt.figure(figsize=(10, 6))
        plt.title("Важность признаков")
        plt.bar(range(len(importances)), importances[indices])
        plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=90)
        plt.tight_layout()
        plt.show()

def plot_confusion_matrix(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Матрица ошибок - {model.best_estimator_["classifier"].__class__.__name__}')
    plt.ylabel('Истинная метка')
    plt.xlabel('Предсказанная метка')
    plt.tight_layout()
    plt.show()


### Подготовка данных

In [57]:
X = data.drop('Mental_Health_Status', axis=1)
y = data['Mental_Health_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = preprocess_data(data)

### Определение моделей и их параметров

In [61]:
models = {
    'Logistic Regression': (LogisticRegression(random_state=42, max_iter=1000),
                            {'classifier__C': [0.1, 1, 10],
                             'classifier__penalty': ['l2', None]}),
    'Decision Tree': (DecisionTreeClassifier(random_state=42),
                      {'classifier__max_depth': [3, 5, 7, 10],
                       'classifier__min_samples_split': [2, 5, 10]}),
    'Stochastic Gradient Descent': (SGDClassifier(random_state=42),
                                    {'classifier__alpha': [0.0001, 0.001, 0.01],
                                     'classifier__loss': ['log_loss', 'hinge', 'modified_huber']}),
    'Gradient Boosting': (GradientBoostingClassifier(random_state=42),
                          {'classifier__n_estimators': [100, 200],
                           'classifier__learning_rate': [0.01, 0.1],
                           'classifier__max_depth': [3, 5]})
}

### Обучение и оценка моделей

In [62]:
results = []
for name, (model, param_grid) in models.items():
    print(f"\nTraining {name}...")
    trained_model = create_and_train_model(preprocessor, model, X_train, y_train, param_grid)
    result = evaluate_model(trained_model, X_test, y_test)
    results.append(result)



Training Logistic Regression...
Fitting 5 folds for each of 6 candidates, totalling 30 fits

Training Decision Tree...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Training Stochastic Gradient Descent...
Fitting 5 folds for each of 9 candidates, totalling 45 fits

Training Gradient Boosting...
Fitting 5 folds for each of 8 candidates, totalling 40 fits


### Отображение результатов

In [63]:
results_df = pd.DataFrame(results)

results_df = results_df.rename(columns={
    'param_classifier__C': 'C',
    'param_classifier__penalty': 'penalty',
    'param_classifier__max_depth': 'max_depth',
    'param_classifier__min_samples_split': 'min_samples_split',
    'param_classifier__n_estimators': 'n_estimators'
})


results_df

,Model,F1 Score,Recall,Precision,C,penalty,max_depth,min_samples_split,param_classifier__alpha,param_classifier__loss,param_classifier__learning_rate,n_estimators
0,LogisticRegression,0.242400,0.2430,0.242992,0.1,l2,NaN,NaN,NaN,NaN,NaN,NaN
1,DecisionTreeClassifier,0.245610,0.2515,0.251866,NaN,NaN,7.0,10.0,NaN,NaN,NaN,NaN
2,SGDClassifier,0.216528,0.2425,0.254322,NaN,NaN,NaN,NaN,0.001,log_loss,NaN,NaN
3,GradientBoostingClassifier,0.263539,0.2655,0.265652,NaN,NaN,5.0,NaN,NaN,NaN,0.01,100.0


### Визуализация результатов

In [ ]:
plt.figure(figsize=(12, 6))
results_df.plot(x='Model', y=['F1 Score', 'Recall', 'Precision'], kind='bar')
plt.title('Model Performance Comparison')
plt.ylabel('Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

###  Анализ лучшей модели


In [ ]:
best_model = results_df.loc[results_df['F1 Score'].idxmax()]
best_model

# Общий вывод об обучении модели

## Сравнение моделей

| Модель | F1 Score | Recall | Precision |
|--------|----------|--------|-----------|
| Логистическая регрессия | 0.24240 | 0.2430 | 0.242992 |
| Дерево решений | 0.24561 | 0.2515 | 0.251866 |

## Анализ результатов

### Производительность
- Обе модели показывают примерно одинаковую производительность
- Дерево решений имеет небольшое преимущество по всем метрикам

### Оценка качества
- Общие показатели производительности для обеих моделей довольно низкие (около 0.25 для всех метрик)
- Это может указывать на:
  - Сложность задачи классификации
    
  - Неспособность моделей уловить важные закономерности в данных

### Параметры моделей
- **Логистическая регрессия**: C = 0.1, penalty = 'l2'
- **Дерево решений**: max_depth = 7, min_samples_split = 10

## Выводы

1. Низкие показатели производительности свидетельствуют о необходимости дальнейшей оптимизации
2. Задача может требовать более глубокого анализа данных и инженерии признаков
3. Небольшое преимущество дерева решений может указывать на наличие нелинейных связей в данных

## Рекомендации по улучшению

1. Провести более детальный анализ данных
2. Рассмотреть использование более сложных моделей (например, Random Forest, Gradient Boosting)
3. Экспериментировать с методами обработки данных и созданием новых признаков
4. Проверить сбалансированность классов и применить методы балансировки при необходимости
5. Расширить диапазон гиперпараметров для оптимизации моделей

## Дальнейшие шаги

1. Провести анализ важности признаков (особенно для дерева решений)
2. Рассмотреть возможность использования ансамблевых методов
3. Оценить производительность на отложенной тестовой выборке для проверки на переобучение

---

В целом, результаты указывают на необходимость дальнейшей работы над моделями и, возможно, более глубокого исследования самих данных для улучшения качества предсказаний в задаче классификации психического здоровья на основе использования технологий.